### Importing all the important Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from dotenv import dotenv_values
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise
from sklearn.metrics import accuracy_score
import statsmodels.formula.api as sm
from scipy.sparse import csr_matrix
import os
import streamlit as sl

### Reading different datasets 

### Filtering Movie titles

### Reading sorted Data

In [ ]:
n= pd.read_csv("../data/movie_sorted.csv")

In [ ]:
n.tail()

### Reading the rating files for movieid 1 and cleaning the file 

In [ ]:
df = pd.read_csv("../data/mv_0000001.txt", names=["customerid","rating","date"] )

In [ ]:
df.head()

### checking the first cleaned rating file 

In [ ]:
df= pd.read_csv("../data/movieid1_rating.csv")

In [ ]:
df.head()

### Creating a function to read and clean all the rating files

### Creating and checking a test version of concatinated rating files

In [ ]:
df_s= pd.read_csv("../data/ratings_small.csv", names=["customerid","rating","date", "movieid"])

In [ ]:
df_s["date"] = pd.to_datetime(df_s["date"], format = "%Y-%m-%d")

In [ ]:
df_s.info()

### Creating the large file containing all the ratings

### Reading and transforming the Qualify  file to add the Machine Learing results

### Reading the cleaned qualify file 

In [ ]:
qualifier= pd.read_csv("../data/qualify.csv")

In [ ]:
qualifier

### Cleaning the probe file to compare the results of qualify dataset created with help of machine learning Algorithms

### Reading the cleaned probe file

In [ ]:
probability = pd.read_csv("../data/probe.csv")

In [ ]:
probability

## NEXT TASKS

#### Filter the data to create a better machine learning program

In [2]:
small= pd.read_csv("../data/ratings_small.csv" )
small.head()

,1488844,3.0,2005-09-06,1
0,822109,5.0,2005-05-13,1
1,885013,4.0,2005-10-19,1
2,30878,4.0,2005-12-26,1
3,823519,3.0,2004-05-03,1
4,893988,3.0,2005-11-17,1


In [3]:
movies = pd.read_csv("../data/movie_sorted.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17770 entries, 0 to 17769
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieid  17770 non-null  int64 
 1   year     17770 non-null  int64 
 2   title    17770 non-null  object
dtypes: int64(2), object(1)
memory usage: 416.6+ KB


In [4]:
ratings = pd.read_csv("../../thymestamps-working-folder/data/_select_ratings_customerid_ratings_rating_ratings_movieid_netfli.csv")
ratings.head()

,customerid,rating,movieid,title
0,2407893,4.0,6974,The Usual Suspects
1,1673319,5.0,6974,The Usual Suspects
2,602,5.0,6974,The Usual Suspects
3,2604811,4.0,6974,The Usual Suspects
4,1329723,5.0,6974,The Usual Suspects


In [ ]:
ratings.info()

In [ ]:
ratings["rating"].isna().sum()

In [ ]:
ratings['customerid'].nunique(), ratings['movieid'].nunique(), ratings['movieid'].max()

In [5]:
user = csr_matrix((ratings['rating'], (ratings['customerid'], ratings['movieid'])))

#### like pivot_table but much more memory efficient

In [6]:
user.shape

(2649430, 17770)

In [7]:
# customerid vector
user[1,:].todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [8]:
# movieid vector
user[:,1].todense().shape

(2649430, 1)

In [ ]:
UU = pairwise.euclidean_distances(user)
print(UU.shape)

sns.heatmap(UU, cmap='coolwarm')

### Deciding a Machine learning technique
#### Neighborhood-based Collaborative Filtering( Nearest Neighbors)
#### Neighborhood Components Analysis
#### nearest neighbors classifier

#### Test the algorithm

#### maybe create some visualisations to check the data distribution 

#### Add ratings in Qualify dataset and compare with probe to check the accuracy

#### Create a Predictor system which takes 3 inputs from user and provides list of related movies 

### Create a GUI using Streamlit